In [5]:
pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 1.7 MB/s eta 0:00:00a 0:00:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

df = pd.read_csv('merged_output.csv')
# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())
with open('negative-words.txt', 'r') as f:
    negative_words = set(f.read().splitlines())
with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    negative_count = len(words.intersection(negative_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > max(positive_count, negative_count):
        return 2  # Depressed
    elif negative_count > positive_count:
        return 1  # Negative
    else:
        return 0  # Positive

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the text data
def encode_text(texts, max_length=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_text(X_train)
test_encodings = encode_text(X_test)

# Convert labels to tensors
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating'):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test accuracy: {accuracy:.4f}')

# Function to predict sentiment
def predict_sentiment(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a negative mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
    "I feel so hopeless and sad all the time.",
    "I'm excited about my new job and looking forward to the future.",
    "I don't know if life is worth living anymore.",
    "I had a great day with my friends and family.",
    "This weather is terrible, and I'm having a bad day.",
    "I'm feeling a bit down today, but I'm sure things will get better."
]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
torch.save(model.state_dict(), 'sentiment_bert_model.pth')
tokenizer.save_pretrained('sentiment_bert_tokenizer')

print("Model and tokenizer saved successfully.")

# To load the model and tokenizer later, you can use:
# loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
# loaded_model.load_state_dict(torch.load('sentiment_bert_model.pth'))
# loaded_tokenizer = BertTokenizer.from_pretrained('sentiment_bert_tokenizer')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3:   0%|                                        | 0/855 [00:00<?, ?it/s]

Evaluating: 100%|█████████████████████████████| 214/214 [02:39<00:00,  1.34it/s]


Test accuracy: 0.9772
Text: I feel so hopeless and sad all the time.
Prediction: The user is in a negative mood. Confidence: 0.96

Text: I'm excited about my new job and looking forward to the future.
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I don't know if life is worth living anymore.
Prediction: The user is in a positive mood. Confidence: 0.99

Text: I had a great day with my friends and family.
Prediction: The user is in a positive mood. Confidence: 0.98

Text: This weather is terrible, and I'm having a bad day.
Prediction: The user is in a negative mood. Confidence: 0.98

Text: I'm feeling a bit down today, but I'm sure things will get better.
Prediction: The user is in a positive mood. Confidence: 0.99

Model and tokenizer saved successfully.


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [5]:
!pip install keras

In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Define a custom Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create datasets
max_len = 128  # You can adjust this value based on your needs
train_dataset = SentimentDataset(X_train.to_numpy(), y_train.to_numpy(), tokenizer, max_len)
test_dataset = SentimentDataset(X_test.to_numpy(), y_test.to_numpy(), tokenizer, max_len)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Training loop
def train_model(model, train_loader, optimizer, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')

# Train the model
train_model(model, train_loader, optimizer, device, epochs=3)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Store predictions and true labels
            predictions.append(logits.argmax(dim=1).cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    # Flatten the lists and return
    return np.concatenate(predictions), np.concatenate(true_labels)

# Evaluate the model
predictions, true_labels = evaluate_model(model, test_loader, device)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f'Test Accuracy: {accuracy:.4f}')

# Classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=['Neutral/Negative', 'Positive', 'Depressed']))

# Function to predict sentiment
def predict_sentiment(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = logits.argmax(dim=1).item()
        confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()

    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a neutral/negative mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
    "I feel so hopeless and sad all the time.",
    "I'm excited about my new job and looking forward to the future.",
    "I don't know if life is worth living anymore.",
    "I had a great day with my friends and family.",
    "This weather is terrible, and I'm having a bad day.",
    "I want to kill myself.",
    "I'm feeling a bit down today, but I'm sure things will get better.",
    "I feel like nothing is going right for me today.",
    "I just can't seem to focus on anything right now.",
    "I'm really struggling to keep up, and it's frustrating.",
]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
model.save_pretrained('sentiment_bert_model')
tokenizer.save_pretrained('sentiment_bert_model')

print("Model and tokenizer saved successfully.")

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the text data
def encode_text(texts, max_length=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_text(X_train)
test_encodings = encode_text(X_test)

# Convert labels to tensors
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating'):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/2:  30%|█████████                     | 259/855 [16:11<47:07,  4.74s/it]

In [ ]:
def predict_sentiment(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a negative mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
        "I feel so hopeless and sad all the time.",
        "I'm excited about my new job and looking forward to the future.",
        "I don't know if life is worth living anymore.",
        "I had a great day with my friends and family.",
        "This weather is terrible, and I'm having a bad day.",
        "I'm feeling a bit down today, but I'm sure things will get better.",
        "I'm really proud of what I accomplished today.",
        "Sometimes I wonder if I'll ever find my way.",
        "I had a productive meeting, but I still have a lot on my plate.",
        "Today was just another ordinary day.",
        "I'm thrilled about my upcoming vacation!",
        "I'm feeling overwhelmed by everything that’s happening.",
        "I'm content with where I am in life right now.",
        "I can't shake off this feeling of dread.",
        "I had a fun time at the park with my friends last weekend."
    ]
for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
torch.save(model.state_dict(), 'sentiment_bert_model.pth')
tokenizer.save_pretrained('sentiment_bert_tokenizer')

print("Model and tokenizer saved successfully.")